# <a id='toc1_'></a>[Olympics](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Olympics](#toc1_)       
  - [Where the Olympics has happened?](#toc1_2_)    
  - [Olympics Medal Tally by Countries Overall](#toc1_3_)    
  - [Which country has won the most medals in each Olympics?](#toc1_4_)    
  - [When has India won the most Medals?](#toc1_5_)    
  - [Olympic Results](#toc1_6_)    
  - [Datasets Merge](#toc1_7_)    
  - [India at the Olympics](#toc1_8_)    
  - [Indian Athletes at the Olympics](#toc1_9_)    
  - [Indian Individual Medal Winners at the Summer Olympics](#toc1_10_)    
  - [Indian Team Olympic Winners](#toc1_11_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [472]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from plottable import ColumnDefinition, Table
from plottable.cmap import normed_cmap
from plottable.plots import image

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)

## <a id='toc1_2_'></a>[Where the Olympics has happened?](#toc0_)

In [473]:
# Read the CSV file into a DataFrame
df_Games = pd.read_csv("Olympics_Games.csv")

# Filter for rows where the edition is "Summer"
df_Summer_Games = df_Games[df_Games['edition'].str.contains("Summer")].sort_values(by='year')

# Select relevant columns
df_Summer_Games = df_Summer_Games[['edition_id', 'edition', 'city', 'year', 'competition_date']]

# Replace missing values in 'competition_date' for specific 'edition_id' values
df_Summer_Games.loc[df_Summer_Games['edition_id'].isin([50, 51, 52]), 'competition_date'] = "Not Held Due to War"

# Display the DataFrame
df_Summer_Games.head()


,edition_id,edition,city,year,competition_date
0,1,1896 Summer Olympics,Athina,1896,6 – 13 April
1,2,1900 Summer Olympics,Paris,1900,14 May – 28 October
2,3,1904 Summer Olympics,St. Louis,1904,1 July – 26 November
3,5,1908 Summer Olympics,London,1908,27 April – 31 October
4,6,1912 Summer Olympics,Stockholm,1912,5 May – 27 July


## <a id='toc1_3_'></a>[Olympics Medal Tally by Countries Overall](#toc0_)

In [474]:
df_Medal = pd.read_csv("Olympic_Games_Medal_Tally.csv")

# Filter for rows where the edition is 'Summer'
df_Medal = df_Medal[df_Medal['edition'].str.contains("Summer")]

df_Medal.head()


,edition,edition_id,year,country,country_noc,gold,silver,bronze,total
0,1896 Summer Olympics,1,1896,United States,USA,11,7,2,20
1,1896 Summer Olympics,1,1896,Greece,GRE,10,18,19,47
2,1896 Summer Olympics,1,1896,Germany,GER,6,5,2,13
3,1896 Summer Olympics,1,1896,France,FRA,5,4,2,11
4,1896 Summer Olympics,1,1896,Great Britain,GBR,2,3,2,7


In [475]:
# Group by 'country' and sum the medals
df_Medal_country = df_Medal.groupby('country').agg({
    'gold': 'sum',
    'silver': 'sum',
    'bronze': 'sum',
    'total': 'sum'
}).sort_values(by='total', ascending=False).reset_index()

# Add rank column based on the total medals
df_Medal_country['Rank'] = df_Medal_country['total'].rank(ascending=False, method='first').astype(int)

# Reorder columns to include Rank and reset the index
df_Medal_country = df_Medal_country[['Rank', 'country', 'gold', 'silver', 'bronze', 'total']]

# Display the DataFrame
df_Medal_country.head()

,Rank,country,gold,silver,bronze,total
0,1,United States,1070,841,744,2655
1,2,Soviet Union,395,319,296,1010
2,3,Great Britain,291,323,313,927
3,4,Germany,237,264,290,791
4,5,France,231,257,285,773


## <a id='toc1_4_'></a>[Which country has won the most medals in each Olympics?](#toc0_)

In [476]:
df_Medal_year = df_Medal.groupby(['country', 'year', 'edition']).agg({
    'gold': 'sum',
    'silver': 'sum',
    'bronze': 'sum',
    'total': 'sum'
}).reset_index().sort_values(by='total', ascending=False)

df_Medal_year.head(10)

,country,year,edition,gold,silver,bronze,total
1261,United States,1904,1904 Summer Olympics,80,85,83,248
1094,Soviet Union,1980,1980 Summer Olympics,80,69,46,195
1277,United States,1984,1984 Summer Olympics,83,61,30,174
473,Great Britain,1908,1908 Summer Olympics,56,51,39,146
1095,Soviet Union,1988,1988 Summer Olympics,55,31,46,132
341,East Germany,1980,1980 Summer Olympics,47,37,42,126
1093,Soviet Union,1976,1976 Summer Olympics,49,41,35,125
1285,United States,2016,2016 Summer Olympics,46,37,38,121
1286,United States,2020,2020 Summer Olympics,39,41,33,113
410,France,1900,1900 Summer Olympics,31,41,40,112


## <a id='toc1_5_'></a>[When has India won the most Medals?](#toc0_)

In [477]:
df_Medal_India = df_Medal.groupby(['country', 'year']).agg({
    'gold': 'sum',
    'silver': 'sum',
    'bronze': 'sum',
    'total': 'sum'
}).reset_index().sort_values(by='total', ascending=False)


df_Medal_India[df_Medal_India['country'] == 'India'].head(10)

,country,year,gold,silver,bronze,total
582,India,2020,1,2,4,7
580,India,2012,0,2,4,6
579,India,2008,1,0,2,3
581,India,2016,0,1,1,2
564,India,1900,0,2,0,2
569,India,1952,1,0,1,2
576,India,1996,0,0,1,1
577,India,2000,0,0,1,1
578,India,2004,0,1,0,1
567,India,1936,1,0,0,1


In [478]:
df_merge = pd.merge(df_Games, df_Medal, on='edition_id', how='left')

# Identify columns with '_x' and '_y' suffixes
columns = df_merge.columns
columns_to_drop = [col for col in columns if col.endswith('_y')]
columns_to_rename = {col: col[:-2] for col in columns if col.endswith('_x')}

# Drop columns with '_y' suffix
df_merge = df_merge.drop(columns=columns_to_drop)

# Rename columns with '_x' suffix to remove the suffix
df_merge = df_merge.rename(columns=columns_to_rename)

# Display the merged DataFrame without duplicate columns
df_merge.head()

,edition,edition_id,edition_url,year,city,country_flag_url,country_noc,start_date,end_date,competition_date,isHeld,country,gold,silver,bronze,total
0,1896 Summer Olympics,1,/editions/1,1896,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,6 April,15 April,6 – 13 April,NaN,United States,11.0,7.0,2.0,20.0
1,1896 Summer Olympics,1,/editions/1,1896,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,6 April,15 April,6 – 13 April,NaN,Greece,10.0,18.0,19.0,47.0
2,1896 Summer Olympics,1,/editions/1,1896,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,6 April,15 April,6 – 13 April,NaN,Germany,6.0,5.0,2.0,13.0
3,1896 Summer Olympics,1,/editions/1,1896,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,6 April,15 April,6 – 13 April,NaN,France,5.0,4.0,2.0,11.0
4,1896 Summer Olympics,1,/editions/1,1896,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,6 April,15 April,6 – 13 April,NaN,Great Britain,2.0,3.0,2.0,7.0


In [479]:
df_Medal_India = df_merge.groupby(['country', 'year', 'city']).agg({
    'gold': 'sum',
    'silver': 'sum',
    'bronze': 'sum',
    'total': 'sum'
}).reset_index().sort_values(by='total', ascending=False)

#convert to int
for col in ['gold', 'silver', 'bronze', 'total']:
    df_Medal_India[col] = df_Medal_India[col].astype(int)

df_final = df_Medal_India[df_Medal_India['country'] == 'India'][['year', 'city', 'gold', 'silver', 'bronze', 'total']]

df_final

,year,city,gold,silver,bronze,total
582,2020,Tokyo,1,2,4,7
580,2012,London,0,2,4,6
579,2008,Beijing,1,0,2,3
581,2016,Rio de Janeiro,0,1,1,2
564,1900,Paris,0,2,0,2
569,1952,Helsinki,1,0,1,2
576,1996,Atlanta,0,0,1,1
577,2000,Sydney,0,0,1,1
578,2004,Athina,0,1,0,1
567,1936,Berlin,1,0,0,1


## <a id='toc1_6_'></a>[Olympic Results](#toc0_)

In [480]:
df_results = pd.read_csv('Olympic_Results.csv')
df_results.head()

,result_id,event_title,edition,edition_id,sport,sport_url,result_date,result_location,result_participants,result_format,result_detail,result_description
0,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,na,"Not so much a competition as a coronation, the..."
1,1626,"Giant Slalom, Women1",1998 Winter Olympics,46,Snowboarding,/editions/46/sports/SBD,9 February 1998,"Mt. Yakebitai, Shiga Kogen, Yamanouchi",31 from 14 countries,"Two runs, total time determined placement.",Gates: 38 / 36Length: 936 mStart Altitude: 196...,The women’s giant slalom was postponed one day...
2,76,"Singles, Men",1976 Winter Olympics,40,Luge,/editions/40/sports/LUG,4 – 7 February 1976,"Kunsteis-Bob- und Rodelbahn, Igls",43 from 15 countries,"Four runs, total time determined placement.",Curves: 14Length: 1220 mStart Altitude: ?Verti...,"Once more, the competitors from East and West ..."
3,962,"1,500 metres, Men",1928 Winter Olympics,30,Speed Skating,/editions/30/sports/SSK,14 February 1928 — 9:00,"Olympia-Eisstadion Badrutts Park, St. Moritz",30 from 14 countries,na,na,There was little doubt that the Olympic 1500 m...
4,258824,"Canadian Singles, Slalom, Men",2008 Summer Olympics,53,Canoe Slalom,/editions/53/sports/CSL,11 – 12 August 2008,"Shunyi Aolinpike Shuishang Gongyuan, Mapo, Shunyi",16 from 16 countries,na,na,Two former Olympic champions in the C-1 slalom...


In [481]:
df_Athlete_results = pd.read_csv('Olympic_Athlete_Event_Results.csv')
df_Athlete_results.head()

df_final_results = pd.merge(df_results, df_Athlete_results, on='result_id', how='left')

df_final_results.head()


columns = df_final_results.columns
columns_to_drop = [col for col in columns if col.endswith('_y')]
columns_to_rename = {col: col[:-2] for col in columns if col.endswith('_x')}

# Drop columns with '_y' suffix
df_final_results = df_final_results.drop(columns=columns_to_drop)

# Rename columns with '_x' suffix to remove the suffix
df_final_results = df_final_results.rename(columns=columns_to_rename)

# Display the merged DataFrame without duplicate columns
df_final_results.head()

,result_id,event_title,edition,edition_id,sport,sport_url,result_date,result_location,result_participants,result_format,result_detail,result_description,country_noc,event,athlete,athlete_id,pos,medal,isTeamSport
0,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,na,"Not so much a competition as a coronation, the...",ARM,"Super-Heavyweight, Men",Ashot Danielyan,55118,DNF,NaN,False
1,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,na,"Not so much a competition as a coronation, the...",BUL,"Super-Heavyweight, Men",Velichko Cholakov,104633,3,Bronze,False
2,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,na,"Not so much a competition as a coronation, the...",EGY,"Super-Heavyweight, Men",Mohamed Ehsan,104636,11,NaN,False
3,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,na,"Not so much a competition as a coronation, the...",GUA,"Super-Heavyweight, Men",Joel Bran,104634,13,NaN,False
4,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,na,"Not so much a competition as a coronation, the...",IRI,"Super-Heavyweight, Men",Hossein Reza Zadeh,94941,1,Gold,False


In [482]:
df_bio = pd.read_csv('Olympic_Athlete_Bio.csv')
df_bio.head()

,athlete_id,name,sex,born,height,weight,country,country_noc,description,special_notes
0,65649,Ivanka Bonova,Female,4 April 1949,166.0,55,Bulgaria,BUL,Personal Best: 400 – 53.54 (1980).,NaN
1,112510,Nataliya Uryadova,Female,15 March 1977,184.0,70,Russian Federation,RUS,NaN,Listed in Olympians Who Won a Medal at the Eur...
2,114973,Essa Ismail Rashed,Male,14 December 1986,165.0,55,Qatar,QAT,Personal Best: 10000 – 27:20.97 (2006).,Listed in Olympians Who Won a Medal at the Asi...
3,30359,Péter Boros,Male,12 January 1908,NaN,NaN,Hungary,HUN,"Between 1927 and 1938, Péter Boros competed as...",NaN
4,50557,Rudolf Piowatý,Male,28 April 1900,NaN,NaN,Czechoslovakia,TCH,Rudolf Piowaty joined the Czechoslovak militar...,NaN


## <a id='toc1_7_'></a>[Datasets Merge](#toc0_)

In [483]:
df = pd.merge(df_final_results, df_bio, on='athlete_id', how='left')
df = pd.merge(df, df_Games, on='edition_id', how='left')
columns = df.columns
columns_to_drop = [col for col in columns if col.endswith('_y')]
columns_to_rename = {col: col[:-2] for col in columns if col.endswith('_x')}

# Drop columns with '_y' suffix
df = df.drop(columns=columns_to_drop)

# Rename columns with '_x' suffix to remove the suffix
df = df.rename(columns=columns_to_rename)

# Display the merged DataFrame without duplicate columns
df.head()

,result_id,event_title,edition,edition_id,sport,sport_url,result_date,result_location,result_participants,result_format,...,special_notes,edition_url,year,city,country_flag_url,country_noc,start_date,end_date,competition_date,isHeld
0,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,...,Listed in Doping Irregularities at the Olympic...,/editions/26,2004,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,13 August,29 August,11 – 29 August,NaN
1,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,...,NaN,/editions/26,2004,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,13 August,29 August,11 – 29 August,NaN
2,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,...,Listed in Olympians Who Won a Medal at the Med...,/editions/26,2004,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,13 August,29 August,11 – 29 August,NaN
3,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,...,Listed in Olympians Who Won a Medal at the Cen...,/editions/26,2004,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,13 August,29 August,11 – 29 August,NaN
4,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,...,Listed in Olympians Who Set a World Record in ...,/editions/26,2004,Athina,https://olympedia-flags.s3.eu-central-1.amazon...,GRE,13 August,29 August,11 – 29 August,NaN


In [484]:
df.columns

Index(['result_id', 'event_title', 'edition', 'edition_id', 'sport',
       'sport_url', 'result_date', 'result_location', 'result_participants',
       'result_format', 'result_detail', 'result_description', 'country_noc',
       'event', 'athlete', 'athlete_id', 'pos', 'medal', 'isTeamSport', 'name',
       'sex', 'born', 'height', 'weight', 'country', 'description',
       'special_notes', 'edition_url', 'year', 'city', 'country_flag_url',
       'country_noc', 'start_date', 'end_date', 'competition_date', 'isHeld'],
      dtype='object')

## <a id='toc1_8_'></a>[India at the Olympics](#toc0_)

In [485]:
# Extract the year from the 'edition' column
df['year'] = df['edition'].str.extract(r'(\d{4})')

# Clean the column names
df.columns = df.columns.str.strip()

# Normalize the 'country' column
df['country'] = df['country'].str.strip()

# Select the specific columns and filter for country 'india'
df_cleaned = df[['event_title', 'edition', 'sport', 'event', 'athlete_id', 'name', 'sex', 'born', 'country', 'description', 'special_notes', 'isTeamSport', 'medal', 'year', 'city']]
df_india = df_cleaned[df_cleaned['country'] == 'India']

df_india = df_india[df_india['edition'].str.contains("Summer")]

df_india = df_india.fillna('-')

# Display the cleaned DataFrame for country 'India'
df_india.sort_values(by='year', ascending=False).head(5)

,event_title,edition,sport,event,athlete_id,name,sex,born,country,description,special_notes,isTeamSport,medal,year,city
233221,"Hockey, Women",2020 Summer Olympics,Hockey,"Hockey, Women",147731,Reena Khokhar,Female,10 April 1993,India,-,-,True,-,2020,Tokyo
44227,"Sporting Pistol, 25 metres, Women",2020 Summer Olympics,Shooting,"Sporting Pistol, 25 metres, Women",121001,Rahi Sarnobat,Female,30 October 1990,India,-,Listed in Olympians Who Won a Medal at the Bri...,False,-,2020,Tokyo
48916,"Hockey, Men",2020 Summer Olympics,Hockey,"Hockey, Men",143770,Dilpreet Singh,Male,12 November 1999,India,-,-,True,Bronze,2020,Tokyo
48915,"Hockey, Men",2020 Summer Olympics,Hockey,"Hockey, Men",2503215,Vivek Prasad,Male,25 February 2000,India,-,-,True,Bronze,2020,Tokyo
174476,"100 metres, Women",2020 Summer Olympics,Athletics,"100 metres, Women",130540,Dutee Chand,Female,3 February 1996,India,-,Listed in Olympians Who Won a Medal at the Sum...,False,-,2020,Tokyo


## <a id='toc1_9_'></a>[Indian Athletes at the Olympics](#toc0_)

In [486]:
df_india_individual = df_india[df_india['isTeamSport'] == False]

df_india_individual = df_india_individual[['event_title', 'sport', 'event', 'edition', 'year', 'city', 
                                        'athlete_id', 'name', 'sex', 'born', 'country', 'medal']]

df_india_individual = df_india_individual.sort_values(by='year', ascending=False)

df_india_individual.head(5)


,event_title,sport,event,edition,year,city,athlete_id,name,sex,born,country,medal
315737,"100 metres Backstroke, Men",Swimming,"100 metres Backstroke, Men",2020 Summer Olympics,2020,Tokyo,2503223,Srihari Nataraj,Male,16 January 2001,India,-
107092,"Balance Beam, Women",Artistic Gymnastics,"Balance Beam, Women",2020 Summer Olympics,2020,Tokyo,143730,Pranati Nayak,Female,6 April 1995,India,-
38888,"Individual, Open",Equestrian Eventing,"Individual, Open",2020 Summer Olympics,2020,Tokyo,143767,Fouaad Mirza,Male,6 March 1992,India,-
38301,"50 kilometres Race Walk, Men",Athletics,"50 kilometres Race Walk, Men",2020 Summer Olympics,2020,Tokyo,143721,Gurpreet Singh,Male,9 July 1984,India,-
156909,"Middleweight (69-75 kilograms), Men",Boxing,"Middleweight, Men",2020 Summer Olympics,2020,Tokyo,143764,Ashish Kumar,Male,18 July 1994,India,-


## <a id='toc1_10_'></a>[Indian Individual Medal Winners at the Summer Olympics](#toc0_)

In [487]:
df_india_medal = df_india[df_india['medal'] != '-']

df_india_medal = df_india_medal[df_india_medal['isTeamSport'] == False]

df_india_medal = df_india_medal[['year', 'city', 'sport', 'event', 'event_title', 
                                'name', 'sex', 'born', 'medal']]

# Create a custom order for medals
medal_order = {'Gold': 1, 'Silver': 2, 'Bronze': 3}
df_india_medal['medal_order'] = df_india_medal['medal'].map(medal_order)

# Sort by year in descending order and then by medal
df_india_medal = df_india_medal.sort_values(by=['year', 'medal_order'], ascending=[False, True])

# Drop the temporary medal_order column
df_india_medal = df_india_medal.drop(columns=['medal_order'])

#remove the row index
df_india_medal = df_india_medal.reset_index(drop=True)

df_india_medal


,year,city,sport,event,event_title,name,sex,born,medal
0,2020,Tokyo,Athletics,"Javelin Throw, Men","Javelin Throw, Men",Neeraj Chopra,Male,24 December 1997,Gold
1,2020,Tokyo,Weightlifting,"Flyweight, Women","Flyweight (≤49 kilograms), Women",Mirabai Chanu,Female,8 August 1994,Silver
2,2020,Tokyo,Wrestling,"Featherweight, Freestyle, Men","Featherweight, Freestyle (≤57 kilograms), Men",Ravi Kumar,Male,12 December 1997,Silver
3,2020,Tokyo,Badminton,"Singles, Women","Singles, Women",P. V. Sindhu,Female,5 July 1995,Bronze
4,2020,Tokyo,Wrestling,"Welterweight, Freestyle, Men","Welterweight, Freestyle (≤65 kilograms), Men",Bajrang Punia,Male,26 February 1994,Bronze
5,2020,Tokyo,Boxing,"Welterweight, Women","Welterweight (64-69 kilograms), Women",Lovlina Borgohain,Female,2 October 1997,Bronze
6,2016,Rio de Janeiro,Badminton,"Singles, Women","Singles, Women",P. V. Sindhu,Female,5 July 1995,Silver
7,2016,Rio de Janeiro,Wrestling,"Lightweight, Freestyle, Women","Lightweight, Freestyle (≤58 kilograms), Women",Sakshi Malik,Female,3 September 1992,Bronze
8,2012,London,Shooting,"Rapid-Fire Pistol, 25 metres, Men","Rapid-Fire Pistol, 25 metres, Men",Vijay Kumar,Male,19 August 1985,Silver
9,2012,London,Wrestling,"Welterweight, Freestyle, Men","Welterweight, Freestyle (≤66 kilograms), Men",Sushil Kumar,Male,26 May 1983,Silver


In [488]:
df_india_medal.groupby(['year', 'sport']).agg({
    'medal': 'count'
}).reset_index().sort_values(by=['year', 'medal'], ascending=[False, False])

,year,sport,medal
18,2020,Wrestling,2
14,2020,Athletics,1
15,2020,Badminton,1
16,2020,Boxing,1
17,2020,Weightlifting,1
12,2016,Badminton,1
13,2016,Wrestling,1
10,2012,Shooting,2
11,2012,Wrestling,2
8,2012,Badminton,1


## <a id='toc1_11_'></a>[Indian Team Olympic Winners](#toc0_)

In [489]:
df_india_team_medal = df_india[df_india['medal'] != '-']

df_india_team_medal = df_india_team_medal[df_india_team_medal['isTeamSport'] == True]

df_india_team_medal = df_india_team_medal[['year', 'city', 'sport', 'event', 'event_title', 
                                'name', 'sex', 'born', 'medal']]

# Create a custom order for medals
medal_order = {'Gold': 1, 'Silver': 2, 'Bronze': 3}
df_india_team_medal['medal_order'] = df_india_team_medal['medal'].map(medal_order)

# Sort by year in descending order and then by medal
df_india_team_medal = df_india_team_medal.sort_values(by=['year', 'medal_order'], ascending=[False, True])

# Drop the temporary medal_order column
df_india_team_medal = df_india_team_medal.drop(columns=['medal_order'])

#remove the row index
df_india_team_medal = df_india_team_medal.reset_index(drop=True)

df_india_team_medal.head()


,year,city,sport,event,event_title,name,sex,born,medal
0,2020,Tokyo,Hockey,"Hockey, Men","Hockey, Men",Vivek Prasad,Male,25 February 2000,Bronze
1,2020,Tokyo,Hockey,"Hockey, Men","Hockey, Men",Dilpreet Singh,Male,12 November 1999,Bronze
2,2020,Tokyo,Hockey,"Hockey, Men","Hockey, Men",Rupinder Pal Singh,Male,11 November 1990,Bronze
3,2020,Tokyo,Hockey,"Hockey, Men","Hockey, Men",Surender Kumar,Male,23 November 1993,Bronze
4,2020,Tokyo,Hockey,"Hockey, Men","Hockey, Men",Manpreet Singh,Male,26 June 1992,Bronze


In [490]:
df_india_medal = df_india[df_india['medal'] != '-']

df_india_medal = df_india_medal[df_india_medal['isTeamSport'] == False]

df_india_medal = df_india_medal[['name', 'sport', 'event', 'year', 'city', 
                                'medal']]

# Create a custom order for medals
medal_order = {'Gold': 1, 'Silver': 2, 'Bronze': 3}
df_india_medal['medal_order'] = df_india_medal['medal'].map(medal_order)

# Sort by year in descending order and then by medal
df_india_medal = df_india_medal.sort_values(by=['year', 'medal_order'], ascending=[False, True])

# Drop the temporary medal_order column
df_india_medal = df_india_medal.drop(columns=['medal_order'])

#remove the row index
df_india_medal = df_india_medal.reset_index(drop=True)

#rename the column by capitalizing the first letter
df_india_medal = df_india_medal.rename(columns={'name': 'Athlete', 'medal': 'Medal', 'year': 'Year', 'city': 'City', 'sport': 'Sport', 'event': 'Event'})

df_india_medal


,Athlete,Sport,Event,Year,City,Medal
0,Neeraj Chopra,Athletics,"Javelin Throw, Men",2020,Tokyo,Gold
1,Mirabai Chanu,Weightlifting,"Flyweight, Women",2020,Tokyo,Silver
2,Ravi Kumar,Wrestling,"Featherweight, Freestyle, Men",2020,Tokyo,Silver
3,P. V. Sindhu,Badminton,"Singles, Women",2020,Tokyo,Bronze
4,Bajrang Punia,Wrestling,"Welterweight, Freestyle, Men",2020,Tokyo,Bronze
5,Lovlina Borgohain,Boxing,"Welterweight, Women",2020,Tokyo,Bronze
6,P. V. Sindhu,Badminton,"Singles, Women",2016,Rio de Janeiro,Silver
7,Sakshi Malik,Wrestling,"Lightweight, Freestyle, Women",2016,Rio de Janeiro,Bronze
8,Vijay Kumar,Shooting,"Rapid-Fire Pistol, 25 metres, Men",2012,London,Silver
9,Sushil Kumar,Wrestling,"Welterweight, Freestyle, Men",2012,London,Silver


In [491]:
# Define colors
bg_color = "#FFFFFF"  # White background
text_color = "#000000"  # Black text

# Set default matplotlib rcParams for plot styling
plt.rcParams['text.color'] = text_color
plt.rcParams['font.family'] = "monospace"
plt.rcParams['axes.facecolor'] = bg_color
plt.rcParams['figure.facecolor'] = bg_color
plt.rcParams['axes.titleweight'] = 'bold'

In [492]:
col_defs = [
     ColumnDefinition(
        name="Athlete",
        textprops = {"ha" : "left"},
        width=1.2
    ),
     ColumnDefinition(
        name="Sport",
        textprops = {"ha" : "center"},
        width=1.2
    ),
    ColumnDefinition(
        name="Event",
        textprops = {"ha" : "center"},
        width=2
    ),
    ColumnDefinition(
        name="Year",
        textprops = {"ha" : "center"},
        width=0.3
    ),
    ColumnDefinition(
        name="City",
        textprops = {"ha" : "center"},
        width=1
    ),
    ColumnDefinition(
        name="Medal",
        textprops = {"ha" : "center"},
        width=0.5
    )
]

In [493]:
fig, ax = plt.subplots(figsize=(20, 32), dpi=450)

table = Table(
    df_india_medal,
    column_definitions=col_defs,
    index_col="Athlete",
    row_dividers=True,
    row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 5))},
    footer_divider=True,
    textprops={"fontsize": 20, "fontname": "Futura", "color": "#000000"},
    col_label_divider_kw={"linewidth": 1, "linestyle": "-" },
    column_border_kw={"linewidth": 2, "linestyle": "-"},
    ax=ax
).autoset_fontcolors(colnames=["Medal"])

#Row colors
row_colors = {
    "Gold": "#FFD700",
    "Silver": "#E6E8FA",
    "Bronze": "#CD7F32"
}


# Highlight specific rows with colors where Medal Column is Gold Silver or Bronze

highlight_rows = {
    "Gold": df_india_medal[df_india_medal["Medal"] == "Gold"].index,
    "Silver": df_india_medal[df_india_medal["Medal"] == "Silver"].index,
    "Bronze": df_india_medal[df_india_medal["Medal"] == "Bronze"].index
}

for category, indices in highlight_rows.items():
    for idx in indices:
        table.rows[idx].set_facecolor(row_colors[category])

#add logo to the top left
logo = plt.imread('Olympic Rings.png')
logo_ax = fig.add_axes([0.12, 0.85, 0.1, 0.1])
logo_ax.imshow(logo)
logo_ax.axis('off')

header_text = f"Indian Athletes at the Olympics"
subheader_text = f"Data: Kaggle | Viz: @VisualizeTrends"
fig.text(0.528, .9, header_text, color="#3D195C", ha="center", va="center", fontsize=50, fontweight="bold", fontname = 'Trebuchet MS')
fig.text(0.52, 0.885, subheader_text, color = "#3D195C" , ha="center", va="center", fontsize=20, fontname = 'Trebuchet MS')

# Save the figure
fig.savefig("Olympics India.png", bbox_inches="tight", facecolor=bg_color)

plt.show()
